# Parte 1: Extracción de Datos
En esta sección se van a normalizar los datos de la base de datos de `datos_inmobiliarios` por medio de consultas en SQL y además eliminar filas que esten repetidas.
## 1. Homogeneizar campo ciudad:
> **Nota:** La columna ciudad va a cambiarse con `UPDATE` pero se puede crear una columna nueva, con la información y trabajar desde pandas con las columnas correctas

In [ ]:
%%bigquery
SELECT *,
  CASE
    WHEN REGEXP_CONTAINS(LOWER(ciudad), r'bogot[aá]') THEN 'BOGOTÁ'
    ELSE UPPER(TRIM(ciudad))
  END AS ciudad_homogeneizada
FROM `pruebaep-461015.datos_mercado_inmobiliario.dataset_prueba_data`;

Query is running:   0%|          |

Downloading:   0%|          |

,ciudad,id,portal_inmobiliario,codigo_web,tipo_inmueble,tipo_negocio,departamento,subzona,nombre_publicacion,direccion,...,fecha_insercion_third_party,fecha_publicacion,url,descripcion,pisos_edificio,piso,ascensor,fecha_insercion_interna,last_edited,ciudad_homogeneizada
0,BOGOTÁ,cf9d671a-4667-42ca-a843-a427e5f05d74,Fincaraiz,10097277,Apartamento,FOR_SALE,Cundinamarca,Teusaquillo,Apartaestudio en venta,"Bogotá, d.c., Bogotá, Zona chapinero, Teusaqui...",...,2024-09-14,2023-05-15,http://fincaraiz.com.co/apartaestudio-en-venta...,None,<NA>,<NA>,<NA>,2025-03-20 14:07:51.650993+00:00,2025-03-15,BOGOTÁ
1,BOGOTÁ,cf9d671a-4667-42ca-a843-a427e5f05d74,Fincaraiz,10097277,Apartamento,FOR_SALE,Cundinamarca,Teusaquillo,Apartaestudio en venta,"Bogotá, d.c., Bogotá, Zona chapinero, Teusaqui...",...,2024-09-14,2023-05-15,http://fincaraiz.com.co/apartaestudio-en-venta...,None,<NA>,<NA>,<NA>,2025-03-20 14:07:51.650993+00:00,2025-03-15,BOGOTÁ
2,BOGOTÁ,237b8ac0-b63c-4b1c-b6ba-a2296d1353c3,Fincaraiz,10346284,Apartamento,FOR_SALE,Cundinamarca,Teusaquillo,Apartaestudio en venta,"Bogotá, d.c., Bogotá, Zona chapinero, Chapinero",...,2024-09-03,2023-12-05,http://fincaraiz.com.co/apartaestudio-en-venta...,None,<NA>,<NA>,<NA>,2025-03-20 14:07:51.650993+00:00,2025-03-04,BOGOTÁ
3,BOGOTÁ,237b8ac0-b63c-4b1c-b6ba-a2296d1353c3,Fincaraiz,10346284,Apartamento,FOR_SALE,Cundinamarca,Teusaquillo,Apartaestudio en venta,"Bogotá, d.c., Bogotá, Zona chapinero, Chapinero",...,2024-09-03,2023-12-05,http://fincaraiz.com.co/apartaestudio-en-venta...,None,<NA>,<NA>,<NA>,2025-03-20 14:07:51.650993+00:00,2025-03-04,BOGOTÁ
4,BOGOTÁ,82bed303-c6f4-4447-8d81-244cafce5180,Fincaraiz,10601866,Apartamento,FOR_SALE,Cundinamarca,Santa Fé,Apartaestudio en venta,"Bogotá, d.c., Bogotá, Las nieves",...,2024-09-14,2024-02-16,http://fincaraiz.com.co/apartaestudio-en-venta...,None,<NA>,<NA>,<NA>,2025-03-20 14:07:51.650993+00:00,2025-03-15,BOGOTÁ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139995,BOGOTÁ,46745e00-5465-4954-8c4b-afcca0d6657a,Metro Cuadrado,15315-C0001-05,Apartamento,FOR_SALE,Cundinamarca,Usaquén,Proyecto EDIFICIO BALMES SANTA BARBARA,"Bogotá D.C., Norte, Otros, SANTA BARBARA, SANT...",...,2023-09-09,NaT,https://www.metrocuadrado.com/proyecto/edifici...,Edificio de cinco pisos con 32 Apartasuites ...,<NA>,<NA>,<NA>,2025-03-20 14:07:51.650993+00:00,2025-03-15,BOGOTÁ
139996,BOGOTÁ,588f3772-b8e1-4e59-a42e-e2611aa02096,Metro Cuadrado,15315-C0001-07,Apartamento,FOR_SALE,Cundinamarca,Usaquén,Proyecto EDIFICIO BALMES SANTA BARBARA,"Bogotá D.C., Norte, Otros, SANTA BARBARA, SANT...",...,2024-11-21,NaT,https://www.metrocuadrado.com/proyecto/edifici...,Edificio de cinco pisos con 32 Apartasuites ...,<NA>,<NA>,<NA>,2025-03-20 14:07:51.650993+00:00,2025-03-15,BOGOTÁ
139997,BOGOTÁ,588f3772-b8e1-4e59-a42e-e2611aa02096,Metro Cuadrado,15315-C0001-07,Apartamento,FOR_SALE,Cundinamarca,Usaquén,Proyecto EDIFICIO BALMES SANTA BARBARA,"Bogotá D.C., Norte, Otros, SANTA BARBARA, SANT...",...,2024-11-21,NaT,https://www.metrocuadrado.com/proyecto/edifici...,Edificio de cinco pisos con 32 Apartasuites ...,<NA>,<NA>,<NA>,2025-03-20 14:07:51.650993+00:00,2025-03-15,BOGOTÁ
139998,BOGOTÁ,b74b290e-e57b-458d-9f65-c2d2717a18aa,Metro Cuadrado,16918-C0001-06,Apartamento,FOR_SALE,Cundinamarca,Usaquén,Proyecto nahikú,"Bogotá D.C., Otros, SANTA PAULA, SANTA PAULA",...,2024-07-16,NaT,https://www.metrocuadrado.com/proyecto/nahiku/...,Actualmente nahikú ha alcanzado el 88% en ven...,5,<NA>,<NA>,2025-03-20 14:07:51.650993+00:00,2025-03-01,BOGOTÁ


In [ ]:
%%bigquery
UPDATE `pruebaep-461015.datos_mercado_inmobiliario.dataset_prueba_data`
SET ciudad =
  CASE
    WHEN REGEXP_CONTAINS(LOWER(ciudad), r'bogot[aá]') THEN 'BOGOTÁ'
    ELSE UPPER(TRIM(ciudad))
  END
WHERE TRUE;

Query is running:   0%|          |

""


## 2. Ubicar filas repetidas
1. Primero, hay que corroborar la cantidad de filas que estan repetidas. Según el enunciado, existe la posibilidad de que la "unica diferencia" sea la columna `last_edited` Entonces, se realiza el siguiente Query para verificar eso.

In [ ]:
%%bigquery
SELECT count(*) as valores_duplicados
FROM
  (SELECT
    codigo_web,
    id,
    nombre_publicacion,
    COUNT(*) AS repeticiones
  FROM `pruebaep-461015.datos_mercado_inmobiliario.dataset_prueba_data`
  GROUP BY codigo_web, id, nombre_publicacion
  HAVING COUNT(*) > 1) as no_duplicados;

Query is running:   0%|          |

Downloading:   0%|          |

,valores_duplicados
0,69999


In [ ]:
%%bigquery
SELECT count(*) as dup_all_col FROM
  (SELECT
    codigo_web,
    id,
    nombre_publicacion,
    COUNT(DISTINCT last_edited) AS versiones,
    COUNT(*) AS repeticiones
  FROM `pruebaep-461015.datos_mercado_inmobiliario.dataset_prueba_data`
  GROUP BY codigo_web, id, nombre_publicacion
  HAVING COUNT(*) > 1 AND COUNT(DISTINCT last_edited) > 1) as duplicados;

Query is running:   0%|          |

Downloading:   0%|          |

,dup_all_col
0,0


Con este resultado se confirma que todos los elementos duplicados lo estan en cada una de sus columnas. Con lo cual, se procede a eliminar los registros dumplicados.

## 3. Creación de una nueva tabla con las filas no duplicadas

In [ ]:
%%bigquery
DROP TABLE IF EXISTS `pruebaep-461015.datos_mercado_inmobiliario.dataset_prueba_data_no_duplicados`;

CREATE OR REPLACE TABLE `pruebaep-461015.datos_mercado_inmobiliario.dataset_prueba_data_no_duplicados` AS
SELECT *
FROM (
  SELECT *,
         ROW_NUMBER() OVER (
           PARTITION BY codigo_web, id, nombre_publicacion
           ORDER BY last_edited asc -- puedes cambiar esto si no tienes fecha
         ) AS fila
  FROM `pruebaep-461015.datos_mercado_inmobiliario.dataset_prueba_data`
)
WHERE fila = 1;

Query is running:   0%|          |

""


## 4. Exportar data para manejo en python
Finalmente, con la tabla normalizada se procede a exportar el archivo CSV al bucket donde este el otro archivo.

In [ ]:
%%bigquery
EXPORT DATA
  OPTIONS (
    uri = 'gs://datos_mercado/datos_prueba_técnica_normalizados_*.csv',
    format = 'CSV',
    overwrite = true,
    header = true,
    field_delimiter = ','
  )
AS
SELECT *
FROM `pruebaep-461015.datos_mercado_inmobiliario.datos_sin_duplicados`;

Query is running:   0%|          |

""
